(econmt-bayesian)=
# Bayesian Inference

In this chapter, we'll look at how to perform analysis and regressions using Bayesian techniques.

Let's import a few of the packages we'll need first. Two key packages that we'll be using that you might not have seen before are [**PyMC3**](https://docs.pymc.io/), a Bayesian inference package, and [**Bambi**](https://bambinos.github.io/), which stands for *BAyesian Model-Building Interface*—this helps you use **PyMC3** more easily. We'll also use [**ArviZ**](https://arviz-devs.github.io/) for visualisation of the results from Bayesian inference, a package that builds on **Matplotlib**, but this will get automatically installed when you intall **PyMC3**. You should follow the install instructions for **PyMC3** and **Bambi** carefully and, if you're confident with using different Python environments, it's a good idea to spin up a new 'bayes' environment to try them out in. In case you need a refresher, the chapter on {ref}`code-preliminaries` covers basic information on how to install new packages.

The expert Bayesian may wonder, "why these packages?" There are many others available, even just in Python. One reason is that they're simpler to use for beginners while having enormous power for advanced users too, so they're very much in keeping with Python's low floor, high ceiling style. The other is that they're considerably more mature than some of the more recent alternatives based on popular tensor and machine learning packages (though **PyMC3** is also built on an underlying tensor package). Finally, they also have flexible computing back-ends (including Jax and Numba, and including using GPUs), which means that super users can make their models go *extremely fast* should they need to—you can find more information on the speed-ups that are possible [here](https://www.pymc-labs.io/blog-posts/pymc-stan-benchmark/).

This chapter has benefitted from the many Bayesian blogs and forum conversations across the web, in particular the ones [here](https://elevanth.org/blog/2017/11/28/build-a-better-markov-chain/), [here](https://people.duke.edu/~ccc14/sta-663/MCMC.html#markov-chain-monte-carlo-mcmc), [here](https://github.com/fonnesbeck/Bayes_Computing_Course/blob/master/notebooks/Section2_1-MCMC.ipynb), [here](https://stats.stackexchange.com/questions/4700/what-is-the-difference-between-fixed-effect-random-effect-and-mixed-effect-mode), and [here](https://rlhick.people.wm.edu/stories/bayesian_3.html).

Here are our initial imports and settings:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import pymc3 as pm
import arviz as az

# Plot settings
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)
az.style.use("arviz-darkgrid")

# Pandas: Set max rows displayed for readability
pd.set_option("display.max_rows", 23)

# Set seed for random numbers
seed_for_prng = 78557
prng = np.random.default_rng(seed_for_prng)  # prng=probabilistic random number generator
# Turn off warnings
warnings.filterwarnings('ignore')

## Introduction

The biggest difference between the Bayesian and frequentist approaches (that we've seen in the other chapters) is probably that, in Bayesian models, the parameters are not assumed to be fixed but instead are treated as random variables whose uncertainty is described using probability distributions. The data are considered fixed. You might see the 'inverse probability' formulation of a Bayesian model written as $p(\theta | y)$ where the $y$ are the data, and the $\theta$ are the model parameters. An interesting aspect of Bayesianism is that there is just one estimator: Bayes' theorem.

This is a contrast with the frequentist view, which holds that the data are random but the model parameters are fixed, and models are often expressed as functions, for example $f(y | \theta)$. Frequentist inference typically involves deriving estimators for the model parameters, and these are usually created to minimise the bias, minimise the variance, or maximise the efficiency. Not so with Bayesian inference.

As a reminder, Bayes' theorem says that ${\displaystyle p(A\mid B)={\frac {p(B\mid A)p(A)}{p(B)}}}$, where $A$ and $B$ are distinct events, $p(A)$ is the probability of event $A$ happening, and $p(A\mid B)$ is the probability of $A$ happening given $B$ has happened or is true. $p$ could stand in for a single number, a probability density function, or a probability mass function. When dealing with data and model parameters, and ignoring a rescaling factor, this can be written as:

$$
p({\boldsymbol{\theta }}|{\boldsymbol{y}})\propto p({\boldsymbol{y}}|{\boldsymbol{\theta }})p({\boldsymbol{\theta }}).
$$

In these equations:

1. $p({\boldsymbol{\theta}})$ is the prior put on model parameters—what we think the distribution of model parameters will look like.
2. $p({\boldsymbol{y}}|{\boldsymbol{\theta }})$ is the likelihood of this data given a particular set of model parameters.
3. $p({\boldsymbol{\theta }}|{\boldsymbol{y}})$ is the posterior probability of model parameters given the observed data.

Bayesian modelling proceeds as highlighted in the review article, *Bayesian statistics and modelling* {cite:t}`van2021bayesian`:

![The Bayesian research cycle.](https://media.springernature.com/lw685/springer-static/image/art%3A10.1038%2Fs43586-020-00001-2/MediaObjects/43586_2020_1_Fig1_HTML.png?as=webp)

One key strength of the Bayesian approach is that it preserves uncertainty—by construction, it includes the degree of belief you have in a parameter. This makes it especially useful in cases where uncertainty is important because the uncertainties are propagated all the through to the final results. One disadvantage of the Bayesian approach is that it's not always as fast and doesn't always scale well to very large datasets.

Actually estimating $p({\boldsymbol{y}}|{\boldsymbol{\theta }})p({\boldsymbol{\theta }})$ is done computationally, at least for all but the most trivial examples.

The methods used to do this are all some variation on the *Monte Carlo* method, a tool that goes much wider than Bayesian inference. Monte Carlo techniques use random numbers to choose samples that allow you to estimate properties of interest. It can be used to perform numerical integration and is used for discrete particle simulations in fields like physics and chemistry. {cite:t}`rubinstein2016simulation` provides a good introduction to the Monte Carlo methods more broadly, including simulation; {cite:t}`bielajew2001fundamentals` is a very good free textbook on Monte Carlo methods for particle transport physics.

![An illustation of Monte Carlo integration](https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/MonteCarloIntegrationCircle.svg/440px-MonteCarloIntegrationCircle.svg.png)

*Yoderj, Mysid; Wikipedia*

Say we wanted to compute the integral of a 2D shape, a circle, to get an area. In the figure above, random pairs of numbers (say, $x, y$) are chosen within the unit square. By taking the ratio of the number of points that land within the 2D shape (40) to all samples (50), we can get an approximation of the area by computing (area of square) x ratio of samples $= 3.2 \approx \pi$. This type of algorithm is a form of Monte Carlo rejection sampling.

Markov Chain Monte Carlo takes this a step further. A Markov Chain is a stochastic process where the chance of being in a particular state (for example, taking a particular sample) depends only on the previous state. These have equilibrium distributions; essentially a long-run average distribution over states. Markov Chain Monte Carlo finds the distribution you're after (the posterior) by recording states from the chain; the more steps, the more closely the distribution of samples will match the distribution you're trying to find out about. There are nice walkthroughs of MCMC available [here](https://people.duke.edu/~ccc14/sta-663/MCMC.html#markov-chain-monte-carlo-mcmc), [here](https://github.com/fonnesbeck/Bayes_Computing_Course/blob/master/notebooks/Section2_1-MCMC.ipynb), and [here](https://rlhick.people.wm.edu/stories/bayesian_3.html).

There are many algorithms for undertaking MCMC, with perhaps the most famous being the Metropolis-Hastings algorithm. For this, we have a probability density function to sample from, a function $Q$ that quantifies where to make a transition, and $\theta_0$, the first guess of the parameters. Starting from $\theta = \theta_0$, the algorithm boils down to:
1. starting from this point $\theta$, determine its match with the target distribution by estimating $p = p({\boldsymbol{y}}|{\boldsymbol{\theta}})p({\boldsymbol{\theta}})$
2. propose a new sample, $\theta'$, using some transition function $Q$, and check its match to the target (let this match be $p'$)
3. compute the ratio of densities, $\frac{p'}{p}$, as a metric of favourability of the new point
4. generate a random number $r \thicksim \mathcal{U}[0, 1)$ and compare it to the ratio; accept the new point only if $r < \frac{p'}{p}$
5. repeat

The clever trick here is that, over time and with more samples, the points you draw will be from the target (posterior) distribution because, at each loop, we are forcing the samples to stick to the posterior distribution. The set of samples is often called a trace. You can find a nice walkthrough of the Metropolis-Hastings algorithm, with code, [here](https://towardsdatascience.com/from-scratch-bayesian-inference-markov-chain-monte-carlo-and-metropolis-hastings-in-python-ef21a29e25a).

The Metropolis-Hastings algorithm is a good place to start to understand what's going on in computational Bayesian inference, but there are more modern and powerful algorithms available today, most notably Hamiltonian Monte Carlo (HMC), which takes its inspiration from physics (where the Hamiltonian is an operator that returns the total energy of the system). Instead of sampling completely randomly, which can be inefficient, HMC sees the sampler get 'rolled' (launched in a smooth trajectory) along the estimates of the posterior. There's a very good explanation of this approach [here](https://elevanth.org/blog/2017/11/28/build-a-better-markov-chain/), and a 'conceptual introduction to HMC' may be found in {cite:t}`betancourt2018conceptual`.

The most commonly used HMC sampler is the No U-Turn Sampler (NUTS), and this is what we'll be using for most of our examples.

For the rest of this chapter, you won't need to worry about most of this though! The joy of **PyMC3** and **Bambi** is that they take care of some of the hard work for us. Expert users will want to dig further into the details, but these packages make running your first Bayesian inference easier with sensible default settings.

### Fixed vs Random Effects (An Aside)

As this is going to come up in this chapter, it's worth having a quick review of the difference between fixed and random effects in regression-based models. There's a bit of disagreement about what people really mean when they say "fixed effects" or "random effects", so let's define them clearly here.

> Random effects are estimated with partial pooling, while fixed effects are not.

What does this mean? Well, technically, if an effect is assumed to be a realised value of a random variable, it is called a random effect. We could say that, in frequentist land (where parameters have true values), a single fixed effect model might look like:

$$
y = \gamma\cdot d
$$

where $d \in \{0, 1\}$, which could be whether an individual belongs to, say, group 0 or group 1 and $\gamma$ is a coefficient that is to be estimated. $\gamma$ has a single "true" value, and we can admire the value of $\hat{\gamma}$ in a regression table once we have done our estimation.

In contrast, even in a frequentist model, random effects are allowed to vary: they are drawn from a distribution (or, more accurately, their deviations from the true value are drawn from a distribution). This might be written as

$$
y_{i}= w_{i}
$$

and it could be that, instead of a group-specific intercept as in the fixed effects model above, the intercept can take a *range of values* depending on the individual. Although it's not always true, another way of thinking about this is that fixed effects are constant across individuals while random effects can vary.

So what's this business about partial pooling? Imagine we have a situation where one group is under-sampled relative to the other groups. With few data points, we may not be able to precisely estimate fixed effects. However, partial pooling means that, if you do have few data points within a group, the group effect estimate can be partially based on the more abundant data from other groups. Random effects can provide a good compromise between estimating an effect by completely pooling all groups, which masks group-level variation, and estimating an effect for all groups completely separately, which could give imprecise estimates for low-sample groups. For example, when estimating means using a random effects approach, subgroup means are allowed to deviate a bit from the mean of the larger group, but not by an arbitrary amount—those deviations follow a distribution, typically a Gaussian (aka Normal) distribution. That's where the "random" in random effects comes from: the deviations of subgroups from a "parent group" follow the distribution of a random variable.

The rule of thumb when thinking about how many different categories should exist for a variable to get the random, rather than fixed, effects treatment is that there should be more than 5 or 6 levels at a minimum in order to apply random effects—fewer than this and it's hard to estimate the variance around the central estimate. Because you could potentially precisely estimated a fixed effect if you had lots of samples in those 5 or 6 levels, you're more likely to reach for random effects when you have fewer samples per class.

Random effects are the extension of this partial pooling technique to a wide range of situations: including multiple predictors, mixed continuous and categorical variables, and more.

As a conrete example, taken from a [forum post](https://stats.stackexchange.com/questions/4700/what-is-the-difference-between-fixed-effect-random-effect-and-mixed-effect-mode) by Paul of MassMutual, suppose you want to estimate average US household income by ZIP code. You have a large dataset containing observations of household incomes and ZIP codes. Some ZIP codes are well represented in the dataset, but others have only a couple of households.

For your initial model you would most likely take the mean income in each ZIP. This will work well when you have lots of data for all ZIPs, but the estimates for your poorly sampled ZIPs will suffer from high variance. You can mitigate this by using a shrinkage estimator (aka partial pooling), which will push extreme values towards the mean income across all ZIP codes.

But how much shrinkage/pooling should you do for a particular ZIP? Intuitively, it should depend on the following:

1. How many observations you have in that ZIP
2. How many observations you have overall
3. The individual-level mean and variance of household income across all ZIP codes
4. The group-level variance in mean household income across all ZIP codes

Now, if you model ZIP code as a random effect, the mean income estimate in all ZIP codes will be subjected to a statistically well-founded shrinkage, taking into account all of these factors.

Random effects models automatically handle 4, the variability estimation, for all random effects in the model. This is useful, as it would be hard to do manually. Having accounted for (1)–(4), a random effects model is able to determine the appropriate shrinkage for low-sample groups. It can also handle much more complicated models with many different predictors.

**Mixed effects models** are models that combine random and fixed effects.

## Bayesian Modelling: A Simple Example

We're going to set up a very simple example using synthetic data and a known data generating process;

$$
\mu = \alpha + \beta x
$$

where

$$
Y \mid \alpha, \beta, \sigma \stackrel{\text{ind}}{\thicksim} \mathcal{N}(\mu, \sigma^2)
$$

For our prior distributions on the parameters, we will assume:

- $\alpha \thicksim \mathcal{N}(0, 10)$
- $\beta \thicksim \mathcal{N}(0, 10)$
- $\sigma \thicksim \mathcal{N_{+}}(1)$

where, to be clear, these three parameters are scalars.

These priors are known as *weakly informative priors* because they are quite diffuse—the 10th and 90th percentile of a normal distribution with $\sigma=10$ spans from, roughly, -13 to 13. With a weakly informative prior, a reasonably large amount of data will ensure that the prior will not be important and the posterior will hone in on a sensible range. We're also implicitly assuming that the data are normalised so that 1 is a meaningful change and zero a sensible reference point. You may want to, for example, scale by the standard deviation of the data or take logs of (positive) predictors, so that coeficients can be interpreted as percent changes (more on rescaling later).

We know the 'true' values of the parameters, because we're going to run a synthetic example, so we'll set those first.

In [ ]:
# True parameter values
alpha_true, beta_true, sigma_true = 1, 2.5, 1.5

# Size of dataset
size = 100

num_samples = 1000
num_chains = 2

# Predictor variable: random sample
X = prng.standard_normal(size)

# Simulate outcome variable
Y = alpha_true + beta_true * X + prng.standard_normal(size) * sigma_true

### Using PyMC3 to do Bayesian inference

Next, we'll begin the Bayesian modelling part of the code. We'll be using the powerful [**PyMC3**](https://docs.pymc.io/) package {cite:t}`salvatier2016probabilistic`; check out the documentation for tutorials and examples. PyMC3 is likely the most popular package for probabilistic programming in Python, and its computations are built on what was originally a deep learning library. The aim of the package is to enable you to write down models using an intuitive syntax for the data generating process.

Let's see how to specify a model in PyMC3. We'll start by putting `with pm.Model() as linear_model:`, to set the 'context'; effectively to say, 'what follows will be a model that we'll save as `linear_model`. Any variables created within a given model's context will be automatically assigned to that model. Next, we set our priors, giving the parameters meaningful names. Then, we build our model by writing down our expected outcome and the expected likelihood of observations according to that model. Up till now, we're just rewriting in code all the bits we put down mathematically in the previous subsection. Finally, we run the 'trace': this is the part that estimates the posterior distribution by taking samples.

In [ ]:
with pm.Model() as linear_model:
    # Priors for unknown model parameters
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=10)
    sigma = pm.HalfNormal("sigma", sigma=1)

    # Expected value of outcome
    mu = alpha + beta * X

    # Likelihood (sampling distribution) of observations
    # this says, conditional on confounders, we expect normal variation
    Y_obs = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=Y)

    # Sample the posterior
    trace = pm.sample(num_samples, return_inferencedata=True, chains=num_chains)

So what just happened? The 'NUTS sampler' was auto-assigned. NUTS stands for the 'No-U-turn sampler'. This is a just a fancy way of picking the points to sample $p({\boldsymbol{y}}|{\boldsymbol{\theta }})p({\boldsymbol{\theta }})$ to build up a picture of $p({\boldsymbol{\theta }}|{\boldsymbol{y}})$. It's a good default option for sampling.

Let's take a quick look at the model we just estimated using the `model_to_graphviz` method—though note that this is a very simple model, and this method comes into its own in more complex cases.

In [ ]:
pm.model_to_graphviz(linear_model)

We've made some assumptions in assembling this model, and it's a good idea to make these explicit. The first is that the outcome data are independent once controlling for $X$, the second that the outcomes are a linear function of the inputs, and the third that, at a given $X$, the outcome variable $Y$ will vary normally around the average value with a consistent standard deviation $\sigma$. These are the assumptions for Bayesian normal regression.

### Understanding the results from running a Bayesian model

As discussed in the Introduction, the set of samples forms a trace, and this trace got returned by the `pm.sample` method above. We can now look at the trace that we created in the 2 separate chains we ran. Here it is as a standalone object explore:

In [ ]:
trace

But it's much easier to inspect some of what it holds using the pre-configured summaries, which are in table and chart form. First, a plot of the traces.

In [ ]:
az.plot_trace(trace);

This looks like fairly meaningless noise, so let's walk through what we're seeing here. On the left-hand side, we can see the results of the two chains we ran for each of the three parameters. The x-axis is the parameter value, the y-axis is the probability density (no values shown). We're actually looking at the posterior distributions here! We'll come on to whether they make sense or not later, but for now we just know we've got some estimates of the posterior.

On the right-hand side, we have some traces, with the sample number on the x-axis and the parameter value on the y axis. Just looking at these gives our first sense of whether the sampling has gone well or not. Trace plots can give a sense of whether the model converged (you should see a trace that is a random scatter around a mean value), whether the chains are mixing (they should overlap randomly on the plot), and whether there are specific parameters that the model is struggling to estimate. If the sampling is not going well, neither of these will be true and the chains will not converge to occupy a small area, nor will they mix together.

From the above chart, it looks like we successfully created some well-behaved samples and have what appears to be a good estimate of the posterior. 

Now let's look at our actual estimates of these parameters by running the `summary` function:

In [ ]:
az.summary(trace, round_to=2)

In Bayesian inference, the coefficient or parameter estimates are extremely easy to interpret. For $\alpha$, we have a mean of 1.03 with a standard deviation of $\approx 0.15$, and a 94% confidence window between approximately 0.75 and 1.3. HDI stands for highest density interval, and you can choose a different percentage using the `hdi_prob=` keyword. We can produce a visualisation of the parameter estimates:

In [ ]:
az.plot_posterior(trace, var_names=['alpha'], figsize=(8, 5));

This is telling us exactly what it looks like; the estimate is, with good confidence, around 1.

There were a bunch of other numbers in the trace summary; what did they mean? Let's run through them:

- The Gelman-Rubin $\hat{R}$ statistic, `r_hat`: this is a useful metric to determine whether the traces achieved stationarity, but it does require a number of chains (at least four) to make sense as a diagnostic. For a given parameter, the statistic is the ratio of the variance in the samples between the chains to the variance of samples within the chains, ie $\hat{R} = \frac{\text{variance between chains}}{\text{variance within chains}}$. The value of $\hat{R}$ approaches unity if the chains are properly sampling the target distribution and a rough guide is that it should be less than 1.01.
- Effective sample sizes, `ess_bulk` and `ess_tail`: ideally, samples would be independent but MCMC samples are correlated. These statistics give a sense how many *effectively independent* samples we are drawing given that correlation. The statistics are referred to either as effective samples size (ESS) or number of effective samples, $n_\text{eff}$. The bulk version, `ess_bulk`, is the main statistic, while `ess_tail` is the effective sample size for more extreme values of the posterior (by default, the lower and upper 5th percentiles). The rule of thumb is that both should be $>400$.
- The Monte Carlo standard errors (MCSE), `msce_mean` and `mcse_sd`: They are measurements of the standard error of the mean and the standard error of the standard deviation of the sample chains. They provide an estimate as to how accurate the expectation values given from MCMC samples of the mean and standard deviation are. However, some believe these should not be reported.

The most useful numbers (apart from the coefficient estimates), then, are `ess_bulk` and `ess_tail`, and $\hat{R}$. Remember that these statistics are mostly tell us how good the *sampling* is, rather than how appropriate the *model* is.

### Checking the Posterior Distribution

It would be good to compare the posteriors to the priors, to see how far we got away from those weakly informative normal (and, in one case, half-normal) priors we used and how nice the posteriors look. Remember, it's not a good sign if our posterior is very much influenced by our prior (unless we have a reason to have a strong prior).

To do this we need to assemble all of the information on our model, priors, and posteriors in one object, for which there's a couple of extra steps. We'll stash all of the information about our model in an object called `pm_data`. This will also do sampling, so will take a moment to run.

In [ ]:
pm_data = az.from_pymc3(
        prior=pm.sample_prior_predictive(model=linear_model, random_seed=seed_for_prng),
        posterior_predictive=pm.sample_posterior_predictive(trace, model=linear_model, random_seed=seed_for_prng, var_names=["alpha", "beta", "sigma", "Y_obs"]),
        model=linear_model,
    )
# Add in our traces:
pm_data.extend(trace)

Okay, now we can compare our priors and posteriors. Let's do it for just one variable, $\sigma$, for which our prior was a half normal distribution.

In [ ]:
az.plot_dist_comparison(pm_data, var_names=["sigma"], figsize=(8, 5));

We can see here that the weakly informative prior, which has a relatively wide initial span, has been shrunk to a posterior that is in a tight bunch around the coefficient we set in the true data generating process.

Another typical check that is run is a check on whether the model can reproduce the patterns observed in the real data. We can use the plot for posterior/prior predictive checks to do this:

In [ ]:
az.plot_ppc(pm_data, group="posterior", figsize=(8, 5));

While there are lots of diagnostic charts available in [arviz](https://arviz-devs.github.io/arviz/), in this case it's useful to roll our own to check visually how our model predicts the data, taking into account the heterogeneity in our data in a way that the posterior predictive check chart we just saw does not.

First, we'll extract the samples that we already made, but in the shape of the exogenous variable, $X$. We'll then grab all of the possible sampled lines (and check their dimensions using `.shape`). Don't worry too much about the details of what's going on below; momstly it's just reshaping the data a bit so that it fits nicely with the plotting tool, **matplotlib**. A future version of **PyMC3**, simply called **PyMC**, will make this kind of checking easier to do.

In [ ]:
reshape_dimensions = len(X), int(num_samples*num_chains/len(X))
pp_alpha = np.reshape(pm_data["posterior_predictive"]["alpha"], reshape_dimensions)
pp_beta = np.reshape(pm_data["posterior_predictive"]["beta"], reshape_dimensions)
predicted_lines = pp_alpha.T + pp_beta.T * X
predicted_lines.shape

Now we can plot the mean outcomes and the data, along with confidence intervals for the mean outcomes and for all predictions.

In [ ]:
fig, ax = plt.subplots()
ax.plot(X, Y, "o", ms=4, alpha=0.4, label="Data")
ax.plot(X, predicted_lines.mean(axis=0), label="Mean outcomes", alpha=1, lw=0.2, color="k")
az.plot_hdi(
    X,
    predicted_lines,
    ax=ax,
    fill_kwargs={"alpha": 0.5, "color": "orchid", "label": "Mean Outcome 94% HPD"},
)
az.plot_hdi(
    X,
    pm_data["posterior_predictive"]["Y_obs"],
    ax=ax,
    fill_kwargs={"alpha": 0.2, "color": "coral", "label": "Outcome 94% HPD"},
)
ax.set_xlabel("Predictor (X)")
ax.set_ylabel("Outcome (Y)")
ax.set_title("Posterior predictive checks")
ax.legend(ncol=2, fontsize=10);

### Fixed Effects and Categorical Variables As Regressors

How do we go about fitting a model with a dummy or fixed effect using a categorical variable in Bayesian land? (At least when it's an exogenous variable—we'll come back to the case when the outcome variable is discrete later).

Let's take the previous example and modify to include a group-specific intercept for two groups—so a dummy variable that is 0 for group 0, and 1 for group 1. Our model will now be:

$$
\mu = \alpha + \beta x + \gamma d
$$

where

$$
Y \mid \alpha, \beta, \sigma, \gamma \stackrel{\text{ind}}{\thicksim} \mathcal{N}(\mu, \sigma^2)
$$

For our priors, we will assume:

- $\alpha \thicksim \mathcal{N}(0, 10)$
- $\beta \thicksim \mathcal{N}(0, 10)$
- $\sigma \thicksim \mathcal{N_{+}}(1)$ (the half-normal distribution)
- $\gamma \thicksim \mathcal{N}(0.5, 10)$

where

$$
\begin{align}
    f(y;\mu, \sigma) = \left\{\begin{array}{cll}
\sqrt{\frac{2}{\pi\sigma^2}}\,\mathrm{e}^{-(y-\mu)^2/2\sigma^2} &  & y \ge \mu \\[1em]
0 & & \text{otherwise}.
\end{array}\right.
    \end{align}
$$

is the half-normal distribution.

Note that we are giving $\gamma$ a mean prior that is not 0, because we do not expect there to be no influence at all from the group 1 contributions.

Let's create the synthetic data for this model.

In [ ]:
# True parameter values
α_true, β_true, σ_true, γ_true = 1, 2.5, 1.5, 6

# Size of dataset
size = 200

num_samples = 1000
num_chains = 4

# Predictor variables
X_cat = prng.standard_normal(size)
D = prng.binomial(1, 0.4, size)  # This chooses 1 or 0 with 0.4 prob for 1

# Simulate outcome variable
Y_cat = α_true + β_true * X_cat + γ_true*D + prng.standard_normal(size) * σ_true

And now a model much like we did before:

In [ ]:
with pm.Model() as linear_model_with_dummy:
    # Priors for unknown model parameters
    α = pm.Normal("α", mu=0, sigma=10)
    β = pm.Normal("β", mu=0, sigma=10)
    γ = pm.Normal("γ", mu=0, sigma=10)
    σ = pm.HalfNormal("σ", sigma=1)

    # Expected value of outcome
    μ = α + β * X_cat + γ * D

    # Likelihood (sampling distribution) of observations
    Y_cat_obs = pm.Normal("Y_obs", mu=μ, sigma=σ, observed=Y_cat)

    # Sample the posterior
    trace = pm.sample(num_samples, return_inferencedata=True, chains=num_chains)

Now we check the trace summary statistics—we won't look at the trace visually this time, because we know what the answer should be!

In [ ]:
az.summary(trace, round_to=2)

This is close to the parameters we used to generate the data.

## Priors

This section has benefitted from [Probabilistic Programming and Bayesian Methods for Hackers](https://nbviewer.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/tree/master/) and the [prior choice recommendations page](https://github.com/stan-dev/stan/wiki/Prior-Choice-Recommendations) of the Stan wiki.

### Choosing Priors

While **Bambi** (which we'll come to shortly) will choose priors for you, you can override them—and there are many situations where you may wish to do this. And for **PyMC3**, you always have to specify your priors. But how do you choose?

There are two important paradigms to be aware of when choosing priors: *objective* priors, where the data influence the posterior the most, and *subjective* priors, where the economist expresses their views in the form of the prior.

The quintessential example of an *objective prior* is the flat prior, literally a uniform distribution over the entire possible range of the variable that is to be estimated. By choosing this, you are saying that you put equal weight on all possible values. You would very rarely use this in practice, and priors with soft boundaries are generalled preferred over those with hard boundaries.

A *subjective* or *informative* prior, however, would have more probability density or mass on certain values, biasing the final estimate toward that part of the range. You might use this if there were lots of previous evidence for something, or if there were extra information outside of the model that you wanted to incorporate. An informative prior reflects a higher degree of certainty about where the final values might end up, and will take a stronger pull to overcome.

Inbetween these two extremes are weakly informative priors, which are the default in **Bambi** and a good place to start in **PyMC3** models, especially if your data are appropriately normalised.

We've already seen some handy priors such as the normal distribution, half-normal distribution, and Student-t distribution (plus it's half equivalent too). You can find a full list of priors supported by **PyMC3** [here](https://docs.pymc.io/en/v3/api/distributions.html), in the section on distributions. Some priors to check out are:

- Cauchy and half-Cauchy
- Beta
- Gamma
- Binomial (discrete)
- Bernoulli (discrete)
- Poisson (discrete)

If you need a generic prior as a place to start for anything, some people recommend $\mathcal{N}(0, 1)$ or $\text{Student-t}(3,0,1)$, assuming your data is appropriately normalised and could be negative or positive.

### Scaling Data

Aim to keep all the parameters in your model "scale-free". This means adjusting them so that a unit increase is meaningful. This will help in numerous ways, including giving a chance for priors like $\mathcal{N}(0, 1)$ or $\mathcal{N}(0, 10)$ to work.

Some suggested ways of scaling your data could be:

- Scale by the standard deviation of the data via $x \longrightarrow x/\sigma_{x}$, for example, if you were looking at test scores you might divide by the standard deviation of all the test scores.
- In a regression, take logs of (positive-constrained) predictors and outcomes, then the coefficients can be interpreted as elasticities, $x \longrightarrow \ln x$
- If a variable has a typical value, say 4, then scale by that under a logarithm, $x \longrightarrow \ln(x/4)$

## Bayesian Models via Formulae with Bambi

Packages like **PyMC3** give you a lot of flexibility, but you do have to think about what to specify for priors and model setup, even for quite simple and standard Bayesian models. [**Bambi**](https://bambinos.github.io/), BAyesian Model-Building Interface, provides a more user friendly and high level model-building interface that builds on **PyMC3** and is designed to make it easy to fit Bayesian mixed-effects models. Most notably, it comes with a formulae API that allows you to specify your model with a string describing a formula (like **statsmodels** for conventional regression).

Let's see how to specify the model we did earlier only using **Bambi**. As a reminder, the model was

$$
\mu = \alpha + \beta x
$$

where

$$
Y \mid \alpha, \beta, \sigma \stackrel{\text{ind}}{\thicksim} \mathcal{N}(\mu, \sigma^2)
$$

First, though, we need to import **Bambi** and put the data from the first example into a dataframe.


In [ ]:
import bambi as bmb

In [ ]:
df_bambi = pd.DataFrame({"X": X, "Y": Y})

In [ ]:
# Initialise the model; use "-1" at the end to suppress the constant term
model = bmb.Model('Y ~ X', df_bambi)

# Fit the model using 1000 on each of 4 chains
results = model.fit(draws=1000, chains=4)

# Use ArviZ to plot the results
az.plot_trace(results)

# Key summary and diagnostic info on the model parameters
az.summary(results, round_to=2)

This was a a lot easier to run, and it found much the same results! Note that the same set of variables as we saw in the first example are just appearing with different names here—the true values are an intercept of 1, a coefficient on $X$ of 2.5, and a standard deviation of 1.5 (the variable `Y_sigma`) above. We didn't even have to specify priors; **Bambi** chose these for us. Let's see exactly what it did by looking at the `model` variable:

In [ ]:
model

You can see from this description that **Bambi** went for slightly more tight priors than we did, and opted for the (very popular) half-Student T distribution for the standard deviation

- $\text{Intercept} \thicksim \mathcal{N}(1.3, 7.1)$
- $X \thicksim \mathcal{N}(0, 7)$
- $\sigma \thicksim \mathcal{t_{+}}(4.0, 2.8)$ (the half-Student-t distribution)


where the half-Student-t distribution is

$$
\begin{split}    \begin{align}
    f(y;\mu, \sigma) = \left\{\begin{array}{cll}
\frac{\Gamma\left(\frac{\nu+1}{2}\right)}{\Gamma\left(\frac{\nu}{2}\right)\sqrt{\pi \nu \sigma^2}}\left(1 + \frac{(y-\mu)^2}{\nu \sigma^2}\right)^{-\frac{\nu + 1}{2}} &  & y \ge \mu \\[1em]
0 & & \text{otherwise}.
\end{array}\right.
    \end{align}\end{split}
$$

**Bambi** will try to choose sensible priors for you.

### Fixed Effects in Bambi

Models with fixed effects are also easy to implement. When a categorical common effect with $N$ levels is added to a model, by default, it is coded by $N-1$ dummy variables (i.e., reduced-rank coding). To explicitly remove the intercept, add "-1" to the formula string—just like with frequentist regression in **statsmodels**.

**Bambi** will recognise when a variable is of categorical or Boolean type. But, as the Zen of Python says, explicit is better than implicit; we should also specify that a variable is categorical or boolean (a fixed effect) directly in the formula. This is done, just as in **statsmodels**, by enclosing the variable in a "C(...)", for "category". So we'll add `"C(D)"` to the model formula specification.

Let's revisit the example from earlier but implement it directly in **Bambi** and remove the intercept.

In [ ]:
df_bambi_cat = pd.DataFrame({"X": X_cat, "Y": Y_cat, "D": D})
df_bambi_cat["D"] = df_bambi_cat["D"].astype("category")
model_cat = bmb.Model("Y ~ X + C(D)", data=df_bambi_cat)
model_cat

Again, **Bambi** has made slightly different choices but ones that seem reasonable. Let's fit the model and look at the estimated values.

In [ ]:
# Fit the model using 1000 on each of 4 chains
results_cat = model_cat.fit(draws=1000, chains=4)

In [ ]:
az.summary(results_cat)

This is very similar to the estimate using the longer way of describing a model using **PyMC3** alone.

### Specifying Priors in Bambi

Using *Bambi*, we didn't specify the prior at all—it chose for us. So how do we modify or specify a prior should we need to?

You can always specify priors from the full **PyMC3** selection should you need them. The first way is to more vaguely specify them in the form of a dictionary mapping variable names to types, for example

```python
prior = {"condition":"superwide"}
```

which scales the priors of the distribution by 0.8 (the default is `"wide"`, which has a scale of $\sqrt{1/3}$). But we can also specify full priors. Let's change the prior in this model to demonstrate.

In [ ]:
from bambi import Prior

prior = {"D": Prior("Normal", mu=0.5, sigma=10)}
model_cat = bmb.Model("Y ~ X + C(D)", data=df_bambi_cat, priors=prior)
results_cat = model_cat.fit(draws=1000, chains=4)
az.summary(results_cat, round_to=2)

### Mixed Effects Models in Bambi

This section is indebted to an example in the **Bambi** [documentation](https://bambinos.github.io/).

We are going to demonstrate how to perform a random and fixed effects analysis making use of a replication of a study by Strack, Martin & Stepper (1988). The original Strack et al. study tested a facial feedback hypothesis arguing that emotional responses are, in part, driven by facial expressions (rather than expressions always following from emotions). Strack and colleagues reported that participants rated cartoons as more funny when the participants held a pen in their teeth (inducing a smile) than when they held a pen between their lips (inducing a pout). This outcome variable is recorded as `"value"` in the data. The article has been cited over 1,400 times, and has been influential in popularising the view that affective experiences and outward expressions of affective experiences can both influence each other (instead of the relationship being a one-way street from experience to expression). In 2016, a Registered Replication Report (RRR) led by Wagenmakers and colleagues attempted to replicate Study 1 from Strack, Martin, & Stepper (1988) in 17 independent experiments comprising over 2,500 participants.

Here we use the Bambi model-building interface to quickly re-analyse the RRR data using a Bayesian approach and making use of random effects and fixed effects.

Let's pull down the data:

In [ ]:
df_rrr = pd.read_csv("https://github.com/bambinos/bambi/raw/main/docs/notebooks/data/rrr_long.csv")
df_rrr.head()

`"value"` represents the rating of the cartoon, while `"condition"` is an indicator of whethere the participant was made to smile or not. `"uid"` is a unique identifier for each individual. We'll also introduce controls for gender and age, and drop any invalid values.

Now, a purely fixed effects model for this would look like

In [ ]:
bmb.Model("value ~ condition + age + gender", df_rrr, dropna=True)

Note that an intercept was automatically added.

This model has steam-rollered through some potentially useful information. For example, there is a variable `"study"` that captures which study the subject's experiment was performed in. It takes one of 17 values. While we might expect that subjects responses will have the same pattern of effect sizes, it's reasonable to think some features of different studies might vary. 

To capture some of the variation, we can add a random effect to the model. We'll add intercept deviations for each study. What we're saying here is that we'll allow the intercept to be different for each study, as long the values are drawn from a distribution. Let $i$ represent an individual and $j$ a study. The model is

$$
\text{value}_{ij} = \alpha \cdot \text{condition}_{i} + \beta \cdot \text{age}_{i} + \gamma \cdot \text{gender}_{i} + W_j
$$

where $W_j$ is a random effect intercept. Here's how to specify it in **Bambi**; we use the notation `"(1|study)"` to declare that there should be a constant offset for each study drawn from a distribution.

In [ ]:
# Fixed effects and group specific (or random) intercepts for study
model_rrr = bmb.Model("value ~ condition + age + gender + (1|study)", df_rrr, dropna=True)
model_rrr

**Bambi** has chosen to draw the study-level intercepts from a normal distribution, with a prior on the standard deviation of that normal that is itself a half-normal distribution.

In [ ]:
results_rrr = model_rrr.fit(draws=1000, chains=2)

In [ ]:
az.plot_trace(results_rrr,
              compact=True);

We've seen much of the above before, but the posterior that looks a bit different is the one for the intercept based on the study. We see here that it is not one parameter, but 17 deviations, plus a mean that the deviations are relative to, plus a standard deviation. We can see the estimates in the normal way, using `az.summary`.

In [ ]:
az.summary(results_rrr, round_to=2)

Of course, this isn't the only way we can add random effects. As well as study-specific intercepts, we can add study-specific slopes to the model. That is, we’ll assume that the subjects at each research site have a different baseline appreciation of the cartoons (some find the cartoons funnier than others), and that the effect of condition also varies across sites. The equation for this model is

$$
\text{value}_{ij} = \delta_{j} + \alpha_{j} \cdot \text{condition}_{i} + \beta \cdot \text{age}_{i} + \gamma \cdot \text{gender}_{i}
$$

where $\delta_{j}$ is an intercept that depends on the study, and $\alpha_{j}$ is a slope that depends on the study. Let's run this model using the syntax `"(condition|study)"` to apply the slope and intercept based on the study random effect. If we wanted slopes specific to each study without including a study specific intercept, we would write `"value ~ condition + age + gender + (0 + condition | study)"` instead.

In [ ]:
# Fixed effects and group specific (or random) intercepts & slopes for study
model_slope = bmb.Model("value ~ condition + age + gender + (condition|study)", df_rrr, dropna=True)
model_slope

Let's take a look at the diagram of the model, with the results

In [ ]:
results_slope = model_slope.fit(draws=2000, chains=2, target_accept=0.99)

Let's have a look at the extra variables we introduced through this one change:

In [ ]:
az.plot_trace(results_slope,
              var_names=['condition|study_sigma', "condition|study", "1|study", '1|study_sigma']);

Now, as well as having study specific intercepts, we have a slope that characterises how the effect of condition on value varies by study.

We can look at the posterior of condition more closely to determine whether it does affect how funny the study participants found cartoons—and now we're taking lots of extra variation into account. We can also compare the size of this coefficient against other variables in the model to get a rough idea of how substantial this effect is. We'll use the `plot_forest` function to do this:

In [ ]:
az.plot_forest(
    results_slope,
    var_names=["Intercept", "condition", "age", "gender"],
    figsize=(8, 2),
);

So we see at best a weak relationship between the condition of a subject and the value they give, and the coefficient's range includes 0. While this model is just meant to be an example (not a proper analysis), the conclusion you might draw from the above is that the results of the original study don't replicate, and there is *no* effect.

This was just a short tour of what can be achieved using formulae in **Bambi**. Check out the [documentation](https://bambinos.github.io/) for more.

## Bayesian Generalised Linear Models

Just as we can perform a wider variety of regressions using frequentist maximum likelihood methods, for example logit (aka following Fermi-Dirac statistics), probit, and poission models, so too can we perform these regressions using Bayesian methods.

Let's see an example of how to perform a logistic regression using some synthetic data. We're going to examine the propensity of students to stay in education at 18 years of age as a *binary outcome* (0 for leaving education and 1 for staying in it) and see how it is predicted by a measure of parental income as a fraction of the median income, frac. We'll also use a fixed effect for whether their parents are divorced or not (1 or 0 respectively), called div. We'll create our own synthetic data to illustrate the problem.

The model we'll use is a *logit*, assuming that the data generating process goes like this:

$$
\sigma^{-1}(p) = \ln\left(\frac{p}{1-p}\right) = X'\cdot \vec{\beta} = \alpha + \beta \cdot \text{frac} + \gamma \cdot \text{div}
$$

where $p$ is probability of staying in education and $\sigma$ is the "link function" that translates variables and coefficients into a probability. $\ln( p/(1-p))$ is called the log-odds as it is the log of the odds ratio. The odds ratio is the ratio of the probability, $p$, to the complement of the probability, $1-p$. Note that this model implies that the log-odds ratio is modelled by a standard linear regression. These definitions also imply that the link function is

$$
p = \sigma\left(X'\cdot \vec{\beta}\right) = \frac{1}{1 + e^{-X'\cdot \vec{\beta}}}
$$

While $p\in[0, 1]$, $\sigma^{-1}(p) \in (-\infty, \infty)$ so this link function maps the real number line into the interval zero to one.

Of course, we're actually dealing with a *binary* variable here—outcomes can be 0 or 1, and nothing inbetween, so there's one more piece of the puzzle. Conditional on confounders, i.e. given the value of $p$, the chance of the outcome variable $y$ being 0 or 1 is $p$. In other words, $P(Y=1|y) = p$, which is the definition of the probability mass function of the Bernoulli distribution. Taking that approach over all outcomes $y$, we can say that

$$
y_i \thicksim \text{Bernoulli}(p_i)
$$

Let's generate some synthetic data with these properties; first we set the true values of the data generating process.

In [ ]:
beta_true = 10
gamma_true = -2
alpha_true = -6

And now let's generate some synthetic data. We'll just grab random numbers uniformly between 0 and 2x median income, and assume that there is equal chance of parents being divorced or not (ie a balanced class).

In [ ]:
nobs = 100
df_sch = pd.DataFrame({"frac": prng.uniform(0, 2, nobs),
                                "div": prng.integers(0, 2, size=nobs)})

beta_dot_x = df_sch["frac"]*beta_true + df_sch["div"]*gamma_true + alpha_true
p_vec = 1/(1+np.exp(-(beta_dot_x)))
# Now sample from Bernoulli (binomial with n=1)
y_vec = prng.binomial(1, p=p_vec, size=nobs)
df_sch["stay"] = y_vec
df_sch.sample(5, random_state=seed_for_prng)

Now we'll perform Bayesian inference on the model we've constructed. We could write this model out in full using **PyMC3**, but that's quite long-winded and **Bambi** offers an easier syntax to do it via a formulae specification. The only addition to what we've seen already is that we're going to specify the family of link functions. There are plenty available, but in this case we'll want the "Bernoulli" family because we're dealing with a single 0 or 1 outcome.

Let's specify and fit the model:

In [ ]:
model_logit = bmb.Model('stay ~ frac + C(div)', df_sch, family="bernoulli")
model_logit

In [ ]:
results_logit = model_logit.fit(draws=3000, chains=3)

In this case, we know the true values, so we'll create a chart of the coefficients and the highest density interval around them alongside the true values. We'll use **ArviZ**'s `plot_forest` function for this.

In [ ]:
fig, ax = plt.subplots()
az.plot_forest(results_logit, ax=ax)
for i, val in enumerate([gamma_true, beta_true, alpha_true]):
    ax.scatter(val, ax.get_yticks()[i], s=75, color="red", zorder=5, edgecolors="k")
plt.suptitle("Estimated vs true parameter values")
plt.show()

It would be nice to see some example samples from the posterior alongside the original data, to check that the model produces sensible results. For this, we can use the predict function. Rather than predicting the entire density range, we'll just predict some means here (`kind="mean"`) and put them into a separate variable (via `inplace=False`).

In [ ]:
idata_mean = model_logit.predict(results_logit, kind="mean", inplace=False)
idata_mean

Armed with predictions of the posterior distribution, we can now chart the original data alongside the model predictions. In the plot below, you'll notice that the predictions seem to lie along two separate lines. Do you know why?

In [ ]:
dot_transp = 0.4
dot_size = 40
fig, ax = plt.subplots()
ax.scatter(df_sch.loc[df_sch["div"] == 0, "frac"], df_sch.loc[df_sch["div"] == 0, "stay"],
           color="blue", alpha=dot_transp, label="Parents together", s=dot_size)
ax.scatter(df_sch.loc[df_sch["div"] == 1, "frac"], df_sch.loc[df_sch["div"] == 1, "stay"],
           color="green", alpha=dot_transp, label="Parents divorced", s=dot_size)
ax.scatter(df_sch["frac"], idata_mean.posterior.stay_mean.mean(axis=0).mean(axis=0),
           label="posterior mean", color="C1", alpha=dot_transp, s=dot_size)
ax.set_xlabel("Fraction of median income")
ax.set_title("Staying in education or not: posterior predictions")
ax.set_yticks([0, 1])
ax.set_yticklabels(["0: Left education", "1: Stayed in education"])
ax.legend(fontsize=10, loc="lower right")
ax.set_xlim(-0.25, 2.25)
plt.show()

In addition to logit models, **Pymc3** and **Bambi** both support a wide range of other generalised linear models.

## Other Bayesian Packages

Although **PyMC3** and **Bambi** are the most accessible and, likely, most widely-used Bayesian inference packages in Python, they're certainly not the only ones. Here are a selection of others you might wish to check out:

- [PyStan](https://pystan.readthedocs.io/en/latest/), the Python wrapper for popular C++ Bayesian library Stan
- [cmdstanpy](https://github.com/stan-dev/cmdstanpy), another Python wrapper for Stan
- [Pyro](https://pyro.ai/), "deep probabilistic modelling, unifying the best of modern deep learning and Bayesian modelling", built on Facebook's PyTorch deep learning package
- [Bean Machine](https://beanmachine.org/), "A universal probabilistic programming language to enable fast and accurate Bayesian analysis" from Facebook, also built on PyTorch.
- [Tensorflow Probability](https://www.tensorflow.org/probability), "makes it easy to combine probabilistic models and deep learning on modern hardware (TPU, GPU)", built on Google's Tensorflow deep learning package
- [pomegranite](https://pomegranate.readthedocs.io/en/latest/), "implements fast and flexible probabilistic models ranging from individual probability distributions to compositional models such as Bayesian networks and hidden Markov models".